## Identifying the Optimal locations for the ATM coverage area using clustering algorithm

### Overview

Identifying the correct ATM location is a problem of critical importance and optimizing business operations without affecting customer service is very challenging for the banks. In the case of banking services, the location of ATMs must match the service demands (turnaround time for service, accessibility etc) of the customers. Therefore, it is necessary to analyse the geographical data of neighbourhoods so as to reflect the customers' convenience and tolerance for quality of service in a given geographical region.

### Scope

The project scope is to suggest the neighbourhoods for establishing the ATM in the Toronto city based on the foursquare public API and census data. 

### Approach

I propose to utilize the density based clustering algorithm (GDBSCAN) to segment the neighbourhoods based on the location demographics and the preferences for ATM location among customers utilizing the ATM services. 
I will have three datasources: 

1. I will retrieve Foursquare data from their Public API

2. I will use the census block dataset from the data portal of the city of New York City as a grid for the borough of Manhattan.

3. I will use the data from google maps

I would join the data from all the above mentioned sources and clean them for the consumption. I will perform clustering on these data and propose to use the DBSCAN algorithms to cluster the locations. Using the Numerical Heatmap, I would be able to characterize our clusters and rename them accordingly.

Using the clusters and their characteristics,  i would propose the neighbourhoods for establishing the ATM's 